In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport exv2.experiment_results 

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import glasbey



pd.set_option('display.max_columns', None)
sns.set_theme(rc={'figure.figsize':(12, 6)})
sns.set_context("paper")
sns.set_style("whitegrid")

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

palette = glasbey.create_block_palette(
    [4, 3, 3, 2, 2],
    colorblind_safe=True,
    cvd_severity=90
)
sns.set_palette(palette)

dep_focus = ["baseline_vanilla_full","monolith_feature_monolith_full","serverless_feature_serverless_full"]
ms_focus = ["baseline_vanilla_full","obs_feature_object-storage_full","norec_feature_norecommendations_full"]
lable_names = {
    "baseline_vanilla_full": "Microservice Baseline",
    'jvm_jvm-impoove_full': "Runtime Improvement",
    'monolith_feature_monolith_full': "Monolith Variant",
    'norec_feature_norecommendations_full' : "Service Reduction",
    'obs_feature_object-storage_full' : "Infrastructure Service Replacement",
    'serverless_feature_serverless_full' : "Serverless Variant",
}

In [4]:
exr = exv2.experiment_results.ExperimentResults("data/2024-07-*", load_stats_history=True) # type: ignore
RUN_VARS = exv2.experiment_results.ExperimentResults.RUN_VARS # type: ignore

assert("loadgenerator" not in (list(exr.pods['name'].unique())))
assert("unkown" not in (list(exr.pods['instance'].unique())))


/Users/b/projects/teastore_experiments/exv2/experiment_results.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pods = pd.concat(


In [5]:

left = "exp_scale_pausing"
right = "exp_scale_rampup"


failures = exr.stats_aggregated[exr.stats_aggregated["exp_workload"].isin([left,right])].groupby(["exp_branch","exp_workload"])[["Request Count","Failure Count"]].sum()
failures["Failure Rate"] = failures["Failure Count"] / failures["Request Count"]
failures = failures.unstack()
failures["fr"] = failures["Failure Rate"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)
failures = failures.droplevel(1, axis=1).reset_index()[["exp_branch","fr"]]


In [6]:
left = "exp_scale_shaped"
right = "exp_scale_rampup"


latency = exr.stats_history_aggregated[exr.stats_history_aggregated["exp_workload"].isin([left,right])].groupby(["exp_branch","exp_workload"])[["p50", "p95"]].mean().unstack()
latency = latency / 1000
latency["p50_diff"] = latency["p50"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)
latency["p95_diff"] = latency["p95"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)

# get the range from a moderate workload (shaped) to a stress workload (rampup)

latency = latency.droplevel(1, axis=1).reset_index()
latency = latency[["exp_branch","p50_diff","p95_diff"]]
latency

,exp_branch,p50_diff,p95_diff
0,baseline_vanilla_full,0.07 - 5.53,0.25 - 15.72
1,jvm_jvm-impoove_full,0.03 - 1.31,0.10 - 9.81
2,monolith_feature_monolith_full,0.02 - 21.69,0.07 - 40.23
3,norec_feature_norecommendations_full,0.10 - 2.32,0.43 - 8.10
4,obs_feature_object-storage_full,0.06 - 17.12,0.20 - 28.65
5,serverless_feature_serverless_full,0.17 - 5.29,0.59 - 15.53


In [7]:
# AWS cost-model

serverless_price = 0.0000166667  # based on aws lambda price per GB-s  (frankfurt)
memory_second_price = 0.00511 / 1024 /60 # $/MBs based on AWS nfragate memory price per hour (frankfurt) 
vCPU_second_price = 0.04656 / 60 # $/vCPU based on AWS nfragate memory price per hour (frankfurt) 

pod_configuration = {
    "teastore-recommender": {"cpu": 2600, "memory": 1332},
    "teastore-webui": {"cpu": 1300, "memory": 1950},
    "teastore-image": {"cpu": 1300, "memory": 1950},
    "teastore-auth": {"cpu": 585, "memory": 1332},
    'teastore-registry':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-persistence':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-db':{"cpu": 1000, "memory": 1024}, # not set by default ....
    "teastore-all": {"cpu":1950, "memory":2663},
    "auth": {"cpu": 500, "memory": 500},
}

import numpy as np
# we calculate the cost for each pod based on the configuration and the time it was running
def calc_request_based_billing(row):
    if row["type"] == "pod":
        conf = pod_configuration[row["pod_name"]] 
        return conf["memory"] * memory_second_price + np.ceil(conf["cpu"]/1000) * vCPU_second_price
    elif row["type"] == "function":
        return 500*serverless_price
    

def calc_usage_based_billing(row):
    if row["type"] == "pod":
        return row["memory_usage"] * memory_second_price + np.ceil(row["cpu_usage"]) * vCPU_second_price
    elif row["type"] == "function":
        return row["memory_usage"]*serverless_price

pods = exr.pods
pods["pod_name"] = pods["name"].apply(lambda x: "-".join(x.split("-")[0:2]))

pods["type"] = pods["pod_name"].apply(lambda x: "pod" if x.startswith("teastore") else "function" if x.startswith("auth") else "infra")
# ignore infra pods for now
pods = pods[pods["type"].isin(["pod","function"])]
pods_usage = pods.groupby(exr.RUN_VARS+["run_time","name","pod_name","type"])[["memory_usage","cpu_usage"]].sum().reset_index()

pods_usage["requested_cost"] = pods_usage.apply(calc_request_based_billing, axis=1)
pods_usage["used_cost"] = pods_usage.apply(calc_usage_based_billing, axis=1)

pods_mean_cost = pods_usage.groupby(exr.RUN_VARS)[["requested_cost","used_cost"]].sum().reset_index().groupby(["exp_branch","exp_workload"])[["requested_cost","used_cost"]].mean().reset_index()

requests = exr.stats.groupby(["exp_branch","exp_workload"])[["Request Count","Failure Count"]].sum().reset_index() # total request count
requests["rq"] = requests["Request Count"] - requests["Failure Count"]

pods_mean_cost_per_request = pods_mean_cost.merge(requests[["exp_branch","exp_workload","rq"]], on=["exp_branch","exp_workload"])
pods_mean_cost_per_request["requested_cost_per_r"] = (pods_mean_cost_per_request["requested_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents
pods_mean_cost_per_request["used_cost_per_r"] = (pods_mean_cost_per_request["used_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents


In [8]:
pods_mean_cost_per_request

,exp_branch,exp_workload,requested_cost,used_cost,rq,requested_cost_per_r,used_cost_per_r
0,baseline_vanilla_full,exp_scale_fixed,0.049288,0.025028,713,6.912780,3.510179
1,baseline_vanilla_full,exp_scale_pausing,0.444944,0.198142,1827,24.353826,10.845207
2,baseline_vanilla_full,exp_scale_rampup,0.696291,0.323583,219002,0.317938,0.147754
3,baseline_vanilla_full,exp_scale_shaped,0.445703,0.209206,18933,2.354108,1.104982
4,jvm_jvm-impoove_full,exp_scale_fixed,0.037350,0.014933,1268,2.945562,1.177669
5,jvm_jvm-impoove_full,exp_scale_pausing,0.525231,0.242209,1570,33.454174,15.427337
6,jvm_jvm-impoove_full,exp_scale_rampup,0.624432,0.314833,704696,0.088610,0.044676
7,jvm_jvm-impoove_full,exp_scale_shaped,0.466872,0.218236,21378,2.183888,1.020846
8,monolith_feature_monolith_full,exp_scale_fixed,0.010539,0.005240,1181,0.892345,0.443673
9,monolith_feature_monolith_full,exp_scale_pausing,0.143149,0.068368,1129,12.679301,6.055613
